<a href="https://colab.research.google.com/github/andrea-t94/airflow-net/blob/master/research/finetuning/notebooks/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Qwen2.5 on Airflow DAGs

This notebook demonstrates how to fine-tune the **Qwen2.5-1.5B-Instruct** model on a dataset of **Airflow DAGs** using [Unsloth](https://github.com/unslothai/unsloth) — a library that makes LLM fine-tuning 2x faster and uses 50% less memory.

### Please note
**1. This notebook has been developed and tested on Google Colab**, which is the recommended environment for reproduction. Running it in other environments may require modifications to the setup and installation steps.


**2. If you want to fine-tune the model with your own settings**, you will need to modify:
- **`NEW_MODEL_NAME`** in the configuration cell — change the username to your own Hugging Face account (e.g., `your-username/qwen2.5-1.5b-airflow-instruct`)
- **Hugging Face Token** — ensure you have write permissions to push models to your account

## 1. Setup & Installation
We install `unsloth` and other necessary libraries. If you are running this in Google Colab, it will automatically detect the environment and install the correct versions.

In [ ]:
%%capture
import os
import torch


# Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets huggingface_hub

In [2]:
# Verify GPU and PyTorch
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Detected: {gpu_name}")
    # Unsloth optimization info
    major_version, minor_version = torch.cuda.get_device_capability()
    if major_version >= 8:
        print("✅ GPU supports bfloat16 (Ampere or newer). Operations will be faster.")
    else:
        print("ℹ️ GPU is older than Ampere (e.g., T4). Using float16.")
else:
    print("❌ No GPU detected! Please change runtime type to GPU in 'Runtime > Change runtime type'.")

PyTorch version: 2.9.0+cu126
GPU Detected: Tesla T4
ℹ️ GPU is older than Ampere (e.g., T4). Using float16.


## 2. Configuration & Authentication
Log in to Hugging Face to access datasets and push your model.

In [4]:
from huggingface_hub import login

# Try to get token from Colab secrets, otherwise prompt
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token, add_to_git_credential=True)
except:
    print("Please provide your Hugging Face Token (Permissions: Write)")
    login(add_to_git_credential=True)

Please provide your Hugging Face Token (Permissions: Write)


In [5]:
# Project Configuration
BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
DATASET_NAME = "andrea-t94/airflow-dag-dataset"

# Output Model Name (Change username if needed)
NEW_MODEL_NAME = "andrea-t94/qwen2.5-1.5b-airflow-instruct"

# Training Parameters
MAX_SEQ_LENGTH = 4096 # Fits most DAG files
LOAD_IN_4BIT = True   # Enable 4-bit quantization (QLoRA) to save memory

## 3. Load Model with Unsloth
We load the model in 4-bit precision to fit within standard Colab GPU memory (15GB).

In [6]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None, # Auto-detect float16 or bfloat16 based on GPU
    load_in_4bit = LOAD_IN_4BIT,
)

# Add LoRA (Low-Rank Adaptation) adapters
# This allows us to train only a small fraction of parameters (~1%)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,            # LoRA Rank (8, 16, 32, 64 are common)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Enable for long context
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## 4. Load & Format Dataset
We specificy a formatting function to apply the ChatML template (which Qwen uses) to our dataset.

In [7]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset(DATASET_NAME)

# Inspect dataset sizes
print(f"Train size: {len(dataset['train'])}")
if 'eval' in dataset: print(f"Eval size:  {len(dataset['eval'])}")

# Format function for ChatML
# The dataset should have a 'messages' column matching standard chat format
def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        # Apply chat template but do NOT tokenize yet
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/560k [00:00<?, ?B/s]

data/eval-00000-of-00001.parquet:   0%|          | 0.00/574k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7414 [00:00<?, ? examples/s]

...

Generating eval split:   0%|          | 0/412 [00:00<?, ? examples/s]

Train size: 7414
Eval size:  412


Map:   0%|          | 0/7414 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

## 5. Training
Configure the `SFTTrainer`. We use `gradient_accumulation_steps` to simulate a larger batch size.

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset.get("eval"),
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = True, # Set to True to speed up training if sequence len is variable and usually are shorter than max_seq_len
    args = TrainingArguments(
        per_device_train_batch_size = 4,  # Increase if GPU memory allows
        gradient_accumulation_steps = 8,   # effective_batch = per_device_train_batch_size*gradient_accumulation_steps
        warmup_steps = 10,
        max_steps = -1,                   # Set to -1 for full epochs
        num_train_epochs = 3,          
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",             # Use 8-bit optimizer to save memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  
        save_strategy = "steps", # Save checkpoint every epoch
        eval_strategy = "steps", # Check eval loss during training
        eval_steps = 100, # Evaluate every 100 steps             # Disable WandB for simplicity
    ),
)

In [11]:
# Start Training
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,414 | Num Epochs = 3 | Total steps = 696
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## 6. Save & Push to Hub
Unsloth allows saving both LoRA adapters (small) and the full merged model (large). We'll also push a GGUF version for local inference (e.g., with Ollama).

In [ ]:
# 1. Save LoRA Adapters only (Small file size, fast)
model.save_pretrained("lora_adapters")
model.push_to_hub(f"{NEW_MODEL_NAME}-lora", token=True)

# 2. Save Merged Model (Full model, slower but easier to use)
print("Saving merged model... this might take a while.")
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
model.push_to_hub_merged(NEW_MODEL_NAME, tokenizer, save_method = "merged_16bit", token=True)

In [ ]:
# 3. Convert to GGUF (for Ollama/Llama.cpp)
# Options: q4_k_m, q8_0, f16
print("Converting to GGUF...")
model.push_to_hub_gguf(
    NEW_MODEL_NAME,
    tokenizer,
    quantization_method = ["q4_k_m"],
    token = True
)